In [188]:
import nltk
from nltk.cluster.util import cosine_distance
import numpy as np
from nltk.corpus import stopwords
# Similairy grapoh creation and manipulation
import networkx as nx
import re

In [189]:
# Cosine similarity calculates the cosine angle between two non-zero vectors

# Cosine similarity formula (A.B) / (||A||.||B||)

# Nltk.utils cosine_distance()

In [299]:
def read_article(file_name):
    file = open(file_name, "r",encoding="utf8")
    filedata = file.read()
    article = filedata
    sentences = []
    for sentence in re.split('\.\s',article):
        sentences.append(sentence)
    
    return sentences

In [300]:
read_article("test.txt")

["The tale is set during the reign of King Arthur and tells of a young Cornish farmer's son named Jack who is not only strong but so clever he easily confounds the learned with his penetrating wit",
 "Jack encounters a livestock-eating giant called Cormoran (Cornish: 'The Giant of the Sea' SWF:Kowr-Mor-An) and lures him to his death in a pit trap",
 "Jack is dubbed 'Jack the Giant-Killer' for this feat and receives not only the giant's wealth, but a sword and belt to commemorate the event",
 "A man-eating giant named Blunderbore vows vengeance for Cormoran's death and carries Jack off to an enchanted castle",
 'Jack manages to slay Blunderbore and his brother Rebecks by hanging and stabbing them',
 "He frees three ladies held captive in the giant's castle.On a trip into Wales, Jack tricks a two-headed Welsh giant into slashing his own belly open",
 "King Arthur's son now enters the story and Jack becomes his servant",
 'They spend the night with a three-headed giant and rob him in the 

In [301]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [302]:
def gen_sim_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

In [303]:
def generate_summary(file_name, top_n=2):
    stop_words = stopwords.words("english")
    summarize_text = []
    sentences = read_article(file_name)
    
    sentence_similarity_matrix = gen_sim_matrix(sentences, stop_words)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    
    scores = nx.pagerank(sentence_similarity_graph, max_iter=10000)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)  
    
    for i in range(top_n):
        summarize_text.append("".join(ranked_sentences[i][1]))
    print("Summary \n", '. '.join(summarize_text))

In [304]:
generate_summary("test.txt", 5)

Summary 
 The giant holds captive many knights and ladies and a Duke's daughter who has been transformed into a white doe through the power of a sorcerer. In gratitude for having spared his castle, the three-headed giant gives Jack a magic sword, a cap of knowledge, a cloak of invisibility, and shoes of swiftness.On the road, Jack and the Prince meet an enchanted Lady serving Lucifer. Jack defeats and beheads the giant with a trick involving the house's moat and drawbridge. The tale is set during the reign of King Arthur and tells of a young Cornish farmer's son named Jack who is not only strong but so clever he easily confounds the learned with his penetrating wit. Jack is dubbed 'Jack the Giant-Killer' for this feat and receives not only the giant's wealth, but a sword and belt to commemorate the event
